<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# Object follow-Optimized

In this course, we will show how to track objects using JetBot!
We will use a pre-trained neural network trained on [COCO dataset] (http://cocodataset.org) to detect 90 different common objects. These include:

* Person (index 0)
* Cup (index 47)

Import the "ObjectDetector" class, which uses our pre-trained SSD engine.

### Single camera image detection

In [ ]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

Internally, the 'ObjectDetector' class uses the TensorRT Python API to execute the engine we provide. It is also responsible for pre-processing the input of the neural network and parsing the detected objects.

Currently, it only works with engines created with the ``jetbotssd_tensorrt`` package. This package has utilities for converting models from the TensorFlow object detection API to the optimized TensorRT engine. Next, let's initialize the camera. Our detector needs 300x300 pixels of input, so we will set this parameter when creating the camera.

> Internally, the Camera class uses GStreamer to take advantage of Jetson Nano's Image Signal Processor (ISP), and the speed at which images are captured is very impressive.

> And also unloaded a lot of size calculations from the CPU.

In [ ]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

Next, let's use some camera input to execute our network. By default, the ObjectDetector class expects the camera to generate a format of bgr8.

However, if the input format is different, you can override the default preprocessor function. If there are any COCO objects in the camera's field of view, they should now be stored in the detections variable,

# Display detected objects in the text area

we print them out by the code shown below:

In [ ]:
detections = model(camera.value)

print(detections)

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

You can see the label, confidence, and border position of each object detected in each image. In this example there is only one image (our camera).

To print the first object detected in the first image, we can call the following command

>If no object detected, an error may be appear here.

In [ ]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

> If no object detected, an error may be appear here.

# Control the robot to follow the center object

Now, we want the robot to follow the object of the specified class. To do this, we will do the following work

1. Detect objects that match the specified class
2. Select the object closest to the center of the camera's field of view. This is the target object.
3. Guide the robot to move to the target object, otherwise it will drift
4. If robot car encounter obstacles, turn left
We will also create widgets that control the target object label, robot speed and cornering gain, and control the speed of the robot's cornering based on the distance between the target object and the center of the robot's field of view.

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

# collision_model = torchvision.models.alexnet(pretrained=False)
# collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
# collision_model.load_state_dict(torch.load('best_model.pth'))
# device = torch.device('cuda')
# collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    # Image zoomed to 224,224 versus 224,244 obstacle avoidance model
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Create a robot instance of the drive motor.

In [ ]:
from jetbot import Robot

robot = Robot()

### Import the PID driver module, create a PID controller instance, and initialize the corresponding control variables.

In [ ]:
import PID

global follow_speed
follow_speed = 0.5
global turn_gain
turn_gain = 1.7
global follow_speed_pid, follow_speed_pid_model
follow_speed_pid_model = 1
# follow_speed_pid = PID.PositionalPID(3, 0, 0)
follow_speed_pid = PID.PositionalPID(1.5, 0, 0.05)
global turn_gain_pid
turn_gain_pid = PID.PositionalPID(0.15, 0, 0.05)
global object_yservo_pid
object_yservo_pid = PID.PositionalPID(3, 0.5, 0)

When we need to follow people, we can automatically adjust the angle of the PTZ to the elevation angle of Jetbot followers by the following cells.

In [ ]:
from servoserial import ServoSerial
servo_device = ServoSerial() 
# if label_widget.value == 1:
servo_device.Servo_serial_double_control(1, 2048, 2, 2500)

We display all control widgets and connect the network execution function to the camera update.

In [ ]:
from jetbot import bgr8_to_jpeg

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=77, description='tracked label')

display(widgets.VBox([
    widgets.HBox([image_widget]),
    label_widget,
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Calculate the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Calculate the length of a two-dimensional vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Find the detection closest to the center of the image"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    global follow_speed
    global turn_gain
    global follow_speed_pid
    target_value_speed = 0
    #Update image value
    image = change['new']

    # Execute a conflict model to determine if it is blocking
    # execute collision model to determine if blocked
    # collision_output = collision_model(preprocess(image)).detach().cpu()
    # Apply the "softmax" function to normalize the output vector to have a sum of 1 (this makes it a probability distribution)
    # prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    # blocked_widget.value = prob_blocked
    
    # Blocking does not execute the following object detection program, directly return to the next round of data refresh
    # turn left if blocked
    # if prob_blocked > 0.5:
    #     robot.left(0.3)
    #     image_widget.value = bgr8_to_jpeg(image)
    #     return
        
    # compute all detected objects
    detections = model(image)
    # First, mark the non-tracking objects that appear with blue lines.
    # Draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
    # Select the object you want to track, select the value of label_widget,1 is person,
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # Then mark the object to be tracked with green lines.
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (0, 255, 0), 4)

    # otherwise go forward if no target detected
    '''
    blob:https---------A string of tags generated by the blob object in html5 after being assigned to the video tag.
    '''
    if det is None:
        # robot.forward(float(follow_speed))
        robot.stop()
    # otherwsie steer towards target
    else:
        # move robot forward and steer proportional target's x-distance from center
        center = detection_center(det)

        #Follow speed PID adjustment
        follow_speed_pid.SystemOutput = 90000 * (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        if label_widget.value == 1:
            follow_speed_pid.SetStepSignal(30000)
        elif label_widget.value == 53 or label_widget.value == 55: # 53:Apple 55:orange
            follow_speed_pid.SetStepSignal(10000)
        else:
            follow_speed_pid.SetStepSignal(10000)
        follow_speed_pid.SetInertiaTime(0.2, 0.1)

        #Steering gain PID adjustment
        turn_gain_pid.SystemOutput = center[0]
        turn_gain_pid.SetStepSignal(0)
        turn_gain_pid.SetInertiaTime(0.2, 0.1)

        #Limit steering gain to the effective range
        if label_widget.value == 1:
            target_value_turn_gain = 0.6 + abs(turn_gain_pid.SystemOutput)
        elif label_widget.value == 53 or label_widget.value == 55:
            target_value_turn_gain = 0.5 + abs(turn_gain_pid.SystemOutput)
        else:
            target_value_turn_gain = 0.5 + abs(turn_gain_pid.SystemOutput)
        if target_value_turn_gain < 0:
            target_value_turn_gain = 0
        elif target_value_turn_gain > 2:
            target_value_turn_gain = 2

        #Keep the output motor speed within the effective driving range
        target_value_speed = 0.46 + follow_speed_pid.SystemOutput / 90000
        target_value_speedl = target_value_speed + target_value_turn_gain * center[0]
        target_value_speedr = target_value_speed - target_value_turn_gain * center[0]
        if target_value_speedl<0.5:
            target_value_speedl=0
        elif target_value_speedl>1:
            target_value_speedl = 1
        if target_value_speedr<0.5:
            target_value_speedr=0
        elif target_value_speedr>1:
            target_value_speedr = 1

        robot.set_motors(target_value_speedl, target_value_speedr)
        
        if label_widget.value != 1:
            #Vertical angle camera PID adjustment
            object_yservo_pid.SystemOutput =bbox[1]*300+150*(bbox[3] - bbox[1])
            # print('Y-axis object center position value:')
            # print(object_yservo_pid.SystemOutput)
            object_yservo_pid.SetStepSignal(150)
            object_yservo_pid.SetInertiaTime(0.2, 0.1)
            #Limit the vertical angle camera angle to the specified range
            target_value_object_yservo = int(2048 + object_yservo_pid.SystemOutput)
            servo_device.Servo_serial_control(2, target_value_object_yservo)
    # Update image display to widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

Call the following code block to connect the execution function to each camera frame to update.

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

If the robot is not blocked, you can see that the blue box surrounds the detected object and the target object (the object that the robot follows) will be displayed in green.

When the target is discovered, the robot should turn to the target.

You can call the following code block to manually disconnect the camera and stop the robot.

In [ ]:
import time
camera.unobserve_all()
time.sleep(1.0)
robot.stop()